In [7]:
import os
import pandas as pd
import numpy as np
import torch
import timm
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import models
from timm import create_model
from PIL import Image

In [8]:
# Change with path to your dataset path, you may check the exact path using the files tab (folder icon in the sidebar)
train_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/train"
test_dir = "/kaggle/input/vlg-recruitment-24-challenge/vlg-dataset/vlg-dataset/test"

# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),  # Randomly flip the image vertically
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(45),  # Increased rotation angle
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformations
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

OGtrain_dataset = ImageFolder(root=train_dir, transform=transform)
# Assuming 'dataset' is your original dataset
val_size = int(0.2 * len(OGtrain_dataset))  # 20% for validation
train_size = len(OGtrain_dataset) - val_size
train_dataset, val_dataset = random_split(OGtrain_dataset, [train_size, val_size])

# Validate Class to Index Mapping
print("Class to Index Mapping:", OGtrain_dataset.class_to_idx)
print(OGtrain_dataset.classes)

Class to Index Mapping: {'antelope': 0, 'bat': 1, 'beaver': 2, 'blue+whale': 3, 'bobcat': 4, 'buffalo': 5, 'chihuahua': 6, 'cow': 7, 'dalmatian': 8, 'deer': 9, 'dolphin': 10, 'elephant': 11, 'german+shepherd': 12, 'giant+panda': 13, 'giraffe': 14, 'grizzly+bear': 15, 'hamster': 16, 'hippopotamus': 17, 'humpback+whale': 18, 'killer+whale': 19, 'leopard': 20, 'lion': 21, 'mole': 22, 'mouse': 23, 'otter': 24, 'ox': 25, 'persian+cat': 26, 'pig': 27, 'polar+bear': 28, 'raccoon': 29, 'rat': 30, 'seal': 31, 'siamese+cat': 32, 'skunk': 33, 'spider+monkey': 34, 'tiger': 35, 'walrus': 36, 'weasel': 37, 'wolf': 38, 'zebra': 39}
['antelope', 'bat', 'beaver', 'blue+whale', 'bobcat', 'buffalo', 'chihuahua', 'cow', 'dalmatian', 'deer', 'dolphin', 'elephant', 'german+shepherd', 'giant+panda', 'giraffe', 'grizzly+bear', 'hamster', 'hippopotamus', 'humpback+whale', 'killer+whale', 'leopard', 'lion', 'mole', 'mouse', 'otter', 'ox', 'persian+cat', 'pig', 'polar+bear', 'raccoon', 'rat', 'seal', 'siamese+ca

In [9]:


batch_size = 4 # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [10]:
# Training Setup
input_size = 224 * 224 * 3  # Flattened size of input images
class ConvNeXtWithDropout(nn.Module):
    def __init__(self, dropout_rate=0.35, num_classes=2):
        super(ConvNeXtWithDropout, self).__init__()
        
        # Load the pre-trained ConvNeXt model
        self.convnext = timm.create_model('convnext_large', pretrained=True)
        
        # Remove the original classifier (head)
        self.convnext.reset_classifier(0)
        
        # Add custom classifier with dropout
        self.fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),  # Add dropout layer
            nn.Linear(self.convnext.num_features, 512),  # Adjust input to match output features from ConvNeXt
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, num_classes)  # Output layer
        )
        
    def forward(self, x):
        # Get features from ConvNeXt
        x = self.convnext(x)
        
        # Pass through custom fully connected layers
        return self.fc(x)
        
num_classes = len(OGtrain_dataset.classes)
# Example usage:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConvNeXtWithDropout(dropout_rate=0.35, num_classes=num_classes)  # 2 classes, you can change it to your required number of classes

# Move the model to the selected device (GPU if available, else CPU)
model = model.to(device)


criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-3)
for param in model.parameters():
    param.requires_grad = True



In [11]:
# Example: Stop if validation accuracy doesn't improve for 3 consecutive epochs
best_val_accuracy = 0
patience = 30
epochs_without_improvement = 0


In [ ]:

# Cosine Annealing Scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

print("This cell has begin execution")
# Training Loop
num_epochs = 50
model.train()
for epoch in range(num_epochs):

    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}")

# Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Compute average validation loss and accuracy
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    if val_accuracy > best_val_accuracy:
        best_val_accuracy= val_accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement+=1

    if epochs_without_improvement>=patience:
        print("Early stopping")
        break
    
    # Step the scheduler 
    scheduler.step(val_loss)

This cell has begin execution
Epoch [1/50], Training Loss: 2.6482
Validation Loss: 1.2858, Validation Accuracy: 87.63%


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [2/50], Training Loss: 0.9543
Validation Loss: 0.9511, Validation Accuracy: 90.04%
Epoch [3/50], Training Loss: 0.8615
Validation Loss: 0.9121, Validation Accuracy: 91.30%
Epoch [4/50], Training Loss: 0.8241
Validation Loss: 0.8977, Validation Accuracy: 91.56%
Epoch [5/50], Training Loss: 0.8069
Validation Loss: 0.8768, Validation Accuracy: 92.40%
Epoch [6/50], Training Loss: 0.7823
Validation Loss: 0.8810, Validation Accuracy: 92.30%
Epoch [7/50], Training Loss: 0.7639
Validation Loss: 0.8659, Validation Accuracy: 92.77%
Epoch [8/50], Training Loss: 0.7482
Validation Loss: 0.8734, Validation Accuracy: 92.92%
Epoch [9/50], Training Loss: 0.7477
Validation Loss: 0.8856, Validation Accuracy: 92.51%
Epoch [10/50], Training Loss: 0.7377
Validation Loss: 0.8922, Validation Accuracy: 92.09%
Epoch [11/50], Training Loss: 0.7307
Validation Loss: 0.8897, Validation Accuracy: 92.51%
Epoch [12/50], Training Loss: 0.7264
Validation Loss: 0.8652, Validation Accuracy: 93.13%
Epoch [13/50], Tra

In [ ]:
# Prediction
model.eval()
test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]
test_predictions = []

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        predicted_class = torch.argmax(outputs, dim=1).item()
        test_predictions.append((img_name, OGtrain_dataset.classes[predicted_class]))

# Save Predictions
submission = pd.DataFrame(test_predictions, columns=['image_id', 'class'])
submission.to_csv("/kaggle/working/kaggle_resnet50.csv", index=False)